<a href="https://colab.research.google.com/github/pran872/Brain-Network/blob/main/level3_custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This